In [10]:
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime
import pytz # Untuk zona waktu WIB

# ================= 🔧 KONFIGURASI =================
CRED_PATH = "/work/staklimjerukagung-firebase-admin.json"
DB_URL = "https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/"
DB_NODE_PATH = "/auto_weather_stat/id-05/data"

# RENTANG WAKTU TARGET (WIB)
START_TIME_STR = "2025-11-29 03:09:00"
END_TIME_STR   = "2025-11-29 07:46:00"

# ==================================================

if not firebase_admin._apps:
    cred = credentials.Certificate(CRED_PATH)
    firebase_admin.initialize_app(cred, {'databaseURL': DB_URL})

def clean_zeros_in_range():
    ref = db.reference(DB_NODE_PATH)
    
    # 1. Konversi Waktu String ke Unix Timestamp (Key Database)
    # Kita asumsikan Key di database adalah Unix Timestamp (Angka)
    wib = pytz.timezone('Asia/Jakarta')
    
    start_dt = wib.localize(datetime.strptime(START_TIME_STR, "%Y-%m-%d %H:%M:%S"))
    end_dt   = wib.localize(datetime.strptime(END_TIME_STR, "%Y-%m-%d %H:%M:%S"))
    
    # Konversi ke timestamp string (karena key firebase itu string angka)
    start_key = str(int(start_dt.timestamp()))
    end_key   = str(int(end_dt.timestamp()))
    
    print(f"🎯 Target Rentang Waktu: {START_TIME_STR} s.d {END_TIME_STR}")
    print(f"🔑 Key Range: {start_key} s.d {end_key}")
    
    # 2. Ambil Data dalam Rentang Waktu Saja (Query)
    print("⏳ Mengunduh data dalam rentang waktu...")
    snapshot = ref.order_by_key().start_at(start_key).end_at(end_key).get()
    
    if not snapshot:
        print("❌ Tidak ada data dalam rentang waktu tersebut.")
        return

    # 3. Filter Data yang Bernilai 0
    targets = []
    
    for key, val in snapshot.items():
        # Cek Temp
        if 'temperature' in val and (val['temperature'] == 0 or val['temperature'] == 0.0):
            targets.append((key, 'temperature'))
            
        # Cek Hum
        if 'humidity' in val and (val['humidity'] == 0 or val['humidity'] == 0.0):
            targets.append((key, 'humidity'))
            
    if not targets:
        print("✅ Data Bersih! Dalam rentang waktu ini tidak ada nilai 0.")
        return

    print(f"\n🚨 Ditemukan {len(targets)} field bermasalah (Nilai 0) dalam rentang waktu ini.")
    print("   Contoh yang akan dihapus:")
    for t in targets[:3]: print(f"   - Key: {t[0]} | Field: {t[1]}")
    
    # 4. Eksekusi
    confirm = input("\nKetik 'HAPUS' untuk eksekusi: ")
    if confirm != 'HAPUS':
        print("Dibatalkan.")
        return

    print("\n🔥 Mulai Menghapus Field...")
    count = 0
    
    for key, field in targets:
        try:
            # Hapus HANYA field spesifik
            ref.child(key).child(field).delete()
            print(f"   🗑️ Hapus {field} di node {key}")
            count += 1
        except Exception as e:
            print(f"   ❌ Gagal: {e}")

    print(f"✨ Selesai! {count} variabel 0 berhasil dihapus.")

if __name__ == "__main__":
    clean_zeros_in_range()

🎯 Target Rentang Waktu: 2025-11-29 03:09:00 s.d 2025-11-29 07:46:00
🔑 Key Range: 1764360540 s.d 1764377160
⏳ Mengunduh data dalam rentang waktu...

🚨 Ditemukan 542 field bermasalah (Nilai 0) dalam rentang waktu ini.
   Contoh yang akan dihapus:
   - Key: 1764360632 | Field: temperature
   - Key: 1764360632 | Field: humidity
   - Key: 1764360692 | Field: temperature

🔥 Mulai Menghapus Field...
   🗑️ Hapus temperature di node 1764360632
   🗑️ Hapus humidity di node 1764360632
   🗑️ Hapus temperature di node 1764360692
   🗑️ Hapus humidity di node 1764360692
   🗑️ Hapus temperature di node 1764360812
   🗑️ Hapus humidity di node 1764360812
   🗑️ Hapus temperature di node 1764360992
   🗑️ Hapus humidity di node 1764360992
   🗑️ Hapus temperature di node 1764361052
   🗑️ Hapus humidity di node 1764361052
   🗑️ Hapus temperature di node 1764361112
   🗑️ Hapus humidity di node 1764361112
   🗑️ Hapus temperature di node 1764361172
   🗑️ Hapus humidity di node 1764361172
   🗑️ Hapus temperature

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>